In [1]:
import torch
import torch.nn as nn
import torch.nn.parallel
from __future__ import print_function
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os

torch.cuda.set_device(0)

In [2]:
import cv2
import numpy as np
import functions
batchsize = 64

#X_train = bonemarrow_cell()
X_train = functions.bonemarrow_cell()
X_label = torch.LongTensor(np.zeros((X_train.shape[0]),dtype=int))
X_train = torch.FloatTensor(X_train)
train = torch.utils.data.TensorDataset(X_train,X_label)
train_loader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=batchsize)

dataiter = iter(train_loader)

test,test_label = functions.get_test()
test = test[np.argsort(test_label)]
test_label = test_label[np.argsort(test_label)]

Test_label = torch.LongTensor(test_label.astype('int'))
Test = torch.FloatTensor(test)

test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(Test,Test_label), shuffle=False, batch_size=64)

In [3]:
rand = 128
dis = 1

class _netG(nn.Module):
    def __init__(self, isize = 32, nz = 149, nc = 3, ngf = 64, n_extra_layers=0):
        super(_netG, self).__init__()
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        cngf, tisize = ngf//2, 4
        while tisize != isize:
            cngf = cngf * 2
            tisize = tisize * 2

        main = nn.Sequential()
        # input is Z, going into a convolution
        main.add_module('initial.{0}-{1}.convt'.format(nz, cngf),
                        nn.ConvTranspose2d(nz, cngf, 4, 1, 0, bias=False))
        main.add_module('initial.{0}.batchnorm'.format(cngf),
                        nn.BatchNorm2d(cngf))
        main.add_module('initial.{0}.relu'.format(cngf),
                        nn.ReLU(True))

        csize, cndf = 4, cngf
        while csize < isize//2:
            main.add_module('pyramid.{0}-{1}.convt'.format(cngf, cngf//2),
                            nn.ConvTranspose2d(cngf, cngf//2, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(cngf//2),
                            nn.BatchNorm2d(cngf//2))
            main.add_module('pyramid.{0}.relu'.format(cngf//2),
                            nn.ReLU(True))
            cngf = cngf // 2
            csize = csize * 2

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cngf),
                            nn.Conv2d(cngf, cngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cngf),
                            nn.BatchNorm2d(cngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cngf),
                            nn.ReLU(True))

        main.add_module('final.{0}-{1}.convt'.format(cngf, nc),
                        nn.ConvTranspose2d(cngf, nc, 4, 2, 1, bias=False))
        main.add_module('final.{0}.tanh'.format(nc),
                        nn.Tanh())
        self.main = main

    def forward(self, input):
        return self.main(input)
    
netG = _netG(nz = rand+dis*10)
print (netG)


""" ==================== DISCRIMINATOR  ======================== """

class _netD(nn.Module):
    def __init__(self, isize = 32, nz = 149, nc = 3, ndf = 64, n_extra_layers=0):
        super(_netD, self).__init__()
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        main = nn.Sequential()
        # input is nc x isize x isize
        main.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        main.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            #main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            #nn.BatchNorm2d(cndf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            main.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            #main.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            #nn.BatchNorm2d(out_feat))
            main.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
       # main.add_module('final.{0}-{1}.conv'.format(cndf, 1),
       #                 nn.Conv2d(cndf, 1, 4, 1, 0, bias=False))
        self.main = main
        
    def forward(self, input):
        return self.main(input)


netD = _netD(nz = rand+dis*10)
print (netD)

_netG (
  (main): Sequential (
    (initial.138-256.convt): ConvTranspose2d(138, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (initial.256.batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (initial.256.relu): ReLU (inplace)
    (pyramid.256-128.convt): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.128.batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.128.relu): ReLU (inplace)
    (pyramid.128-64.convt): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.64.batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.64.relu): ReLU (inplace)
    (final.64-3.convt): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (final.3.tanh): Tanh ()
  )
)
_netD (
  (main): Sequential (
    (initial.conv.3-64): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bi

In [4]:
class _netD_D(nn.Module):
    def __init__(self):
        super(_netD_D, self).__init__()
        self.conv = nn.Conv2d(256, 1, 4, 1, 0, bias=False)
        
    def forward(self, x):
        x = self.conv(x)
        return x
    
class _netD_Q(nn.Module):
    def __init__(self, nd = 10):
        super(_netD_Q, self).__init__()
        # input is Z, going into a convolution
        self.conv = nn.Conv2d(256, 10, 4, 1, 0, bias=False)
        self.softmax = nn.LogSoftmax()

    def forward(self, x):
        x = self.conv(x)
        x = self.softmax(x)
       # x = x.view(64,10)
        return x

netD_D = _netD_D()
netD_Q = _netD_Q()

In [5]:
netD, netG, netD_D, netD_Q = netD.cuda(), netG.cuda(), netD_D.cuda(), netD_Q.cuda()

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

netG.apply(weights_init)
netD.apply(weights_init)
netD_Q.apply(weights_init)
netD_D.apply(weights_init)

_netD_D (
  (conv): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
)

In [6]:
optimizerD = optim.Adam([
                {'params': netD.parameters()},
                {'params': netD_D.parameters()}
            ], 2e-4, betas=(0.5, 0.9))

optimizerG = optim.Adam(netG.parameters(), 2e-4, betas=(0.5, 0.9))
 
optimizerQ = optim.Adam([
                {'params': netG.parameters()},            
                {'params': netD.parameters()},
                {'params': netD_Q.parameters()},
            ], 2e-4, betas=(0.5, 0.9))

input = torch.FloatTensor(batchsize, 3, 32, 32)
noise = torch.FloatTensor(batchsize, rand+10*dis,1 ,1 )

fixed_noise = torch.FloatTensor(np.random.multinomial(batchsize, 10*[0.1], size=1))
c = torch.randn(batchsize, 10)
z = torch.randn(batchsize, rand)

label = torch.FloatTensor(1)

real_label = 1
fake_label = 0

criterion = nn.BCELoss()
criterion_logli = nn.NLLLoss(size_average=True)
criterion_mse = nn.MSELoss()

criterion, criterion_logli, criterion_mse = criterion.cuda(), criterion_logli.cuda(), criterion_mse.cuda()
input, label = input.cuda(), label.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
z, c = z.cuda(), c.cuda()

In [7]:
def sample_c(batchsize):
    rand_c = np.zeros((batchsize,10),dtype='float32')
    for i in range(0,batchsize):
        rand = np.random.multinomial(1, 10*[0.1], size=1)
        rand_c[i] = rand
    
    label_c = np.argmax(rand_c,axis=1)
    label_c = torch.LongTensor(label_c.astype('int'))
    rand_c = torch.from_numpy(rand_c.astype('float32'))
    return rand_c,label_c

def zero_grad():
    netD.zero_grad()
    netD_Q.zero_grad()
    netD_D.zero_grad()
    netG.zero_grad()

def weight_clamp():
    for p in netD.parameters():
            p.data.clamp_(-0.01, 0.01)
    for p in netD_D.parameters():
            p.data.clamp_(-0.01, 0.01)
    #for p in netD_Q.parameters():
            #p.data.clamp_(-0.01, 0.01)
    #for p in netD_Q_2.parameters():
            #p.data.clamp_(-0.01, 0.01)
    #for p in netD_Q_3.parameters():
            #p.data.clamp_(-0.01, 0.01)
        
def generate_fix_noise(dis=1, cont=4, rand=128):
    
    fixed_z = np.random.randn(10,rand).repeat(10,axis=0)
    changing_dis = np.zeros((100,10),dtype = np.float32)
    list = [n for n in range(0,10)]*10
    for i in range(0,100):
        changing_dis[i,list[i]] = 1
    fixed_cont = np.zeros((100,cont),dtype = np.float32)
    map1 = np.concatenate((changing_dis,fixed_cont,fixed_z),axis=1)
    
    lst = [map1.astype(np.float32)]
    single_cont = np.asarray([float(n-5)*2/5 for n in range(0,10)]*10,dtype = np.float32)
    
    fixed_dis = np.zeros((100,10),dtype=np.float32)
    for t in range(0,5):
        fixed_dis[t*20:t*20+20,t*2] = 1
        
    for t in range (0,4):
        fixed_cont = np.zeros((100,cont),dtype = np.float32)
        fixed_cont[:,t] = single_cont
        map2 = np.concatenate((fixed_dis,fixed_cont,fixed_z),axis=1)
        lst.append(map2.astype(np.float32))
    
    return lst

def fix_noise(dis=1, rand=128):
        
    fixed_z = np.random.randn(10,rand).repeat(10,axis=0)
    changing_dis = np.zeros((100,10),dtype = np.float32)
    list = [n for n in range(0,10)]*10
    for i in range(0,100):
        changing_dis[i,list[i]] = 1
    map1 = np.concatenate((changing_dis,fixed_z),axis=1)
    
    lst = [map1.astype(np.float32)]
    return lst[0].reshape(100,138,1,1)


one = torch.FloatTensor([1])
mone = one * -1
one = one.cuda()
mone = mone.cuda()
fixed_noise = torch.from_numpy(fix_noise()).cuda()

In [8]:
def calc_gradient_penalty(netD_D, netD, real_data, fake_data,lamda,batch_size):
    #print real_data.size()
    alpha = torch.rand(batch_size,1,1,1)
    #print (real_data.size())
    alpha = alpha.expand(real_data.size())
    alpha = alpha.cuda()

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)
    interpolates = interpolates.cuda()
    interpolates = Variable(interpolates, requires_grad=True)

    disc_interpolates = netD_D(netD(interpolates))#.view(batch_size,-1)

    gradients, = autograd.grad(outputs=disc_interpolates.sum(), inputs=interpolates,
                              create_graph=True)
    
    #gradients, = autograd.grad(outputs=disc_interpolates.sum(), inputs=interpolates,
                              #grad_outputs=torch.ones(disc_interpolates.size()).cuda(),
                              #create_graph=True, retain_graph=True, only_inputs=True)
    #gradients*gradients
    
    gradient_penalty = ((gradients.view(batch_size,-1).norm(2, dim=1) - 1) ** 2).mean()* lamda
    return gradient_penalty

'''
def state_one():
def state_two():
def state_three():
'''

'\ndef state_one():\ndef state_two():\ndef state_three():\n'

In [ ]:
import torch.autograd as autograd

gen_iterations = 0
lamda = 10

for epoch in range(100000):

    dataiter = iter(train_loader)
    i = 0
    
    while i < len(train_loader):
        
        for p in netD.parameters(): 
            p.requires_grad = True 
        for p in netD_D.parameters(): 
            p.requires_grad = True 

        for iter_d in range(0,5):
            if i >=len(train_loader):
                continue
                                
            image_, _ = dataiter.next()
            _batchsize = image_.size(0)
            image_ = image_.cuda()
            i +=1
            input.resize_as_(image_).copy_(image_)
            inputv = Variable(input)
            
            #train with real
            errD_real = netD_D(netD(inputv)).mean()
            errD_real.backward(mone)
            
            # train with fake
            rand_c,label_c = sample_c(_batchsize)
            rand_c = rand_c.cuda()
            c.resize_as_(rand_c).copy_(rand_c)
            z.resize_(_batchsize, rand, 1, 1).normal_(0, 1)
            noise = torch.cat([c,z],1)
            noise_resize = noise.view(_batchsize,rand+10*dis,1,1)
            noisev = Variable(noise_resize, volatile = True)
            fake = Variable(netG(noisev).data)
            inputv = fake
            errD_fake = netD_D(netD(inputv)).mean()
            errD_fake.backward(one)
            
            # train with gradient penalty
            gradient_penalty = calc_gradient_penalty(netD_D,netD, input, fake.data,lamda,_batchsize)
            gradient_penalty.backward()
            
            D_cost = -errD_real + errD_fake + gradient_penalty
            optimizerD.step()
            
            # update Q
            zero_grad()
            noisev = Variable(noise_resize)
            G_sample = netG(noisev)
            Q_c_given_x = netD_Q(netD(G_sample)).view(_batchsize, 10)
            crossent_loss = criterion_logli(Q_c_given_x ,Variable(label_c.cuda()))
            
            mi_loss = 1*crossent_loss
            mi_loss.backward()
            optimizerQ.step()

    # update G  
        for p in netD.parameters(): 
            p.requires_grad = False 
        for p in netD_D.parameters(): 
            p.requires_grad = False 

        zero_grad()
        rand_c,label_c = sample_c(batchsize)
        rand_c = rand_c.cuda()
        c.resize_as_(rand_c).copy_(rand_c)
        z.resize_(batchsize, rand, 1, 1).normal_(0, 1)
        noise = torch.cat([c,z],1)
        noise_resize = noise.view(batchsize,rand+10*dis,1,1)
        noisev = Variable(noise_resize)
        fake = netG(noisev)
        errG = netD_D(netD(fake)).mean()
        errG.backward(mone)
        optimizerG.step()
        gen_iterations += 1
        
        
        # update Q
        
        for p in netD.parameters(): 
            p.requires_grad = True 
        for p in netD_D.parameters(): 
            p.requires_grad = True 

        zero_grad()
        noisev = Variable(noise_resize)
        G_sample = netG(noisev)
        Q_c_given_x = netD_Q(netD(G_sample)).view(batchsize, 10)
        crossent_loss = criterion_logli(Q_c_given_x ,Variable(label_c.cuda()))
            
        mi_loss = 1*crossent_loss
        mi_loss.backward()
        optimizerQ.step()

        
       # print (Q_c_given_x)

        # ent_loss = torch.mean(-torch.sum(c * torch.log(c + 1e-8), dim=1))
       # ent_loss_2 = torch.mean(-torch.sum(c2 * torch.log(c2 + 1e-8), dim=1))
       # ent_loss_3 = torch.mean(-torch.sum(c3 * torch.log(c3 + 1e-8), dim=1))
        
        if gen_iterations % 20 == 0 :
            #errD = D_real - D_fake
            with open("output_cell.txt","w") as f:
                f.write('{0} {1} {2} {3}'.format(epoch, gen_iterations , -D_cost.data[0] , mi_loss.data[0]) + '\n')
            print ('{0} {1} {2} {3}'.format(epoch, gen_iterations , -D_cost.data[0] , mi_loss.data[0]))
            G_sample = netG(Variable(fixed_noise))
            vutils.save_image(G_sample.data, 'fake_cell_2.png',nrow=10,normalize=True)
            
            #vutils.save_image(G_sample.data, '{0}fake_samples_{1}.png'.format(-errD.data[0], gen_iterations))
            #vutils.save_image(G_sample.data, 'fake_samples.png',normalize = True)
            
            #for t in range(0,5):
                #fixed_noise = generate_fix_noise(dis, cont, rand)[t].reshape(100,rand+dis*10+cont,1,1)
                #G_sample = netG(Variable(torch.FloatTensor(fixed_noise).cuda()))
                #vutils.save_image(G_sample.data, 'map_%d_cell.png'% (t),nrow=10,normalize=True)

            #torch.save(netG.state_dict(), './params/0517/tumor_netG_epoch_%d.pth' % (epoch))
            #torch.save(netD.state_dict(), './params/0517/tumor_netD_epoch_%d.pth' % (epoch))
            #torch.save(netD_D.state_dict(), './params/0517/tumor_netD_D_epoch_%d.pth' % (epoch))
            #torch.save(netD_Q.state_dict(), './params/0517/tumor_netD_Q_epoch_%d.pth' % (epoch))
            #torch.save(netD_Q_2.state_dict(), './params/0517/tumor_netD_Q_2_epoch_%d.pth' % (epoch))
            #torch.save(netD_Q_3.state_dict(), './params/0517/tumor_netD_Q_3_epoch_%d.pth' % (epoch))
        
        #storage = np.zeros((100,3,64,64),dtype=np.float32)
        #z_fix = Variable(torch.randn(1,128,1,1).cuda().normal_(0, 1))

1 20 8.35248279572 0.000232604332268
3 40 7.25627851486 0.00568279623985
4 60 6.74875640869 3.22125852108e-05
6 80 5.8663482666 0.0014715497382
7 100 5.6945977211 0.00132467038929
9 120 5.96909952164 0.0291027277708
10 140 5.19147205353 8.23475420475e-05
12 160 6.34396791458 0.000300081446767
13 180 4.54677724838 0.000263751484454
15 200 5.72915220261 0.00091986451298
16 220 5.21468687057 1.33911380544e-05
18 240 5.01160573959 0.00291320122778
19 260 1.93066036701 0.00401925062761
21 280 4.23214960098 0.000524769537151
23 300 3.35070109367 2.80662206933e-05
24 320 3.78325533867 0.000827711541206
26 340 3.462474823 0.00016435328871
27 360 3.88543987274 0.000359685975127
29 380 3.95492196083 7.61919654906e-05
30 400 4.4666018486 0.00199199980125
32 420 5.03944969177 0.000562960747629
33 440 4.07128620148 6.22128136456e-05
35 460 4.0890879631 0.000214081257582
36 480 3.72262740135 0.0359534956515
38 500 3.79451155663 2.36947089434e-05
39 520 3.08243894577 0.00250709801912
41 540 3.7960262

In [11]:
torch.save(netG.state_dict(), 'netG_cell_1_epoch_%d.pth' % (epoch))
torch.save(netD.state_dict(), 'netD_cell_1_epoch_%d.pth' % (epoch))
torch.save(netD_D.state_dict(), 'netD_D_cell_1_epoch_%d.pth' % (epoch))
torch.save(netD_Q.state_dict(), 'netD_Q_cell_1_epoch_%d.pth' % (epoch))

In [15]:
import cv2
import numpy as np
batchSize = 1

X_train = functions.bonemarrow_cell()
#X_train = X_train[0:1000,:,:,:]
X_label = torch.LongTensor(np.zeros((X_train.shape[0]),dtype=int))
X_train_ = torch.FloatTensor(X_train)
train = torch.utils.data.TensorDataset(X_train_,X_label)
dataloader = torch.utils.data.DataLoader(train, shuffle=False, batch_size=batchSize)

dataiter = iter(dataloader)
ngpu = 0
nz = 100
ngf = 64
ndf = 64
nc = 3
n_extra_layers = 0

In [24]:
feature_dict = {}
data_iter = iter(dataloader)
for i,data in enumerate(data_iter):
    feature_dict[i] = np.argmax ( netD_Q(netD(Variable(data[0].cuda()))).data.cpu().numpy() )
    
#feature = np.zeros((max(feature_dict.keys()),4096),dtype=np.float32)
#for i,data in enumerate(feature):
    #data[:] = feature_dict[i] 

In [28]:
np.array(feature_dict.values())

array([6, 4, 2, ..., 9, 9, 5])

In [29]:
arg = [np.argwhere(np.array(feature_dict.values())==n) for n in range(0,10)]
for i in range(0,10):
    cluster = np.take(X_train, arg[i],axis=0).reshape(-1,3,32,32)
    vutils.save_image(torch.from_numpy(cluster), 'cluster%d_%d.png'% (10,i),normalize=True)